# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [31]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [33]:
# Output File (CSV)
output_data_file = "../Images/output_data/cities.csv"

# Convert it to dataframe and clean
cities_df = pd.read_csv(output_data_file)
weather_df = cities_df.drop(columns=["Unnamed: 0"])
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,tiksi,71.69,128.87,4.86,93,83,4.36,RU,1603797496
1,bluff,-46.60,168.33,44.01,87,1,4.00,NZ,1603797496
2,new norfolk,-42.78,147.06,51.01,67,74,1.86,AU,1603797496
3,khatanga,71.97,102.50,4.06,94,97,12.08,RU,1603797496
4,jamestown,42.10,-79.24,42.01,100,90,3.36,US,1603797427


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [34]:
gmaps.configure(api_key=g_key)

In [36]:
locations = weather_df[["Lat", "Lng"]]

# Get the humidity for the heatmap
humidity = weather_df["Humidity"].astype(float)
max_humidity = humidity.max()
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=max_humidity, point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [37]:
weather_df_narrowed_df = weather_df.loc[(weather_df["Max Temp"] > 70) & (weather_df["Max Temp"] < 80) & 
                                        (weather_df["Cloudiness"] == 0)]

weather_df_narrowed_df = weather_df_narrowed_df.dropna(how='any')
weather_df_narrowed_df.reset_index(inplace=True)
del weather_df_narrowed_df["index"]
weather_df_narrowed_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,carnarvon,-24.87,113.63,73.4,78,0,25.28,AU,1603797499
1,meghradzor,40.61,44.65,73.4,29,0,1.12,AM,1603797501
2,east london,-33.02,27.91,78.8,69,0,16.11,ZA,1603797505
3,ballitoville,-29.54,31.21,77.0,73,0,16.11,ZA,1603797521
4,cermik,38.14,39.45,71.6,21,0,5.82,TR,1603797536


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [38]:
hotel_list = []
# Set hotel_df to be the narrowed_df
hotel_df = weather_df_narrowed_df

# Time to go through each city and grab the hotel within 5000 meters and store first hotel
for i in range(len(hotel_df)):
    lat = hotel_df.loc[i]["Lat"]
    lng = hotel_df.loc[i]["Lng"]
    
    parameters = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types": "hotel",
        "key": g_key
    }
    
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(url, parameters)
    json_conversion = response.json()
    try:
        hotel_list.append(json_conversion['results'][0]['name'])
    except:
        hotel_list.append("")

In [39]:
# Store hotel as Hotel Name in hotel_df
hotel_df["Hotel Name"] = hotel_list
hotel_df = hotel_df.dropna(how="any")
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,carnarvon,-24.87,113.63,73.4,78,0,25.28,AU,1603797499,Carnarvon
1,meghradzor,40.61,44.65,73.4,29,0,1.12,AM,1603797501,Meghradzor
2,east london,-33.02,27.91,78.8,69,0,16.11,ZA,1603797505,East London
3,ballitoville,-29.54,31.21,77.0,73,0,16.11,ZA,1603797521,Dolphin Coast
4,cermik,38.14,39.45,71.6,21,0,5.82,TR,1603797536,Çermik


In [40]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [30]:
# Add marker layer ontop of heat map use figure_layout for readability
figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, zoom_level=2, center=(15,25))
hotel_layer = gmaps.marker_layer(
    locations,info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()])
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…